<a href="https://colab.research.google.com/github/nortonvanz/Texas_Airbnb/blob/main/NPS_Texas_Airbnb_c1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivo

Dataset: Rio Airbnb



Objectives - Cycle 1:

Using pyspark:

- 1 Eliminate properties without review
- 2 Identify customers who were detractors in some evaluation
- 3 Get bag of words from these negative reviews

# Cycle 1

## Imports

### PySpark

In [2]:
#pyspark libs and dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
import os
import findspark

#configure JAVA_HOME and SPARK_HOME environment variables in Google Colab, indicating where Java and Apache Spark are installed in the environment
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

#Launch findspark, a Python library that helps you locate the Apache Spark installation on your system and configure the Python environment to interact with Spark.
findspark.init()

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

#Create a Spark session using the PySpark library
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Iniciando com Spark') \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

### Imports

In [6]:
#extract gzip
import gzip
import shutil

#sentiment analysis
from textblob import TextBlob

#User Defined Function (UDF)
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

## Load data

In [7]:
#Upload datasets directly from Inside Airbnb to the Colab temporary environment:
  #Dasets: http://insideairbnb.com/get-the-data/

# listings.csv.gz = Detailed Listings Data
!wget --quiet --show-progress http://data.insideairbnb.com/united-states/tx/austin/2023-09-10/data/listings.csv.gz

# reviews.csv.gz = Detailed Review Data (.gz, contains comments, not on summarized .csv)
!wget --quiet --show-progress http://data.insideairbnb.com/united-states/tx/austin/2023-09-10/data/reviews.csv.gz

listings.csv.gz.1   100%[===================>]   9.99M  26.6MB/s    in 0.4s    
reviews.csv.gz.1    100%[===================>]  53.55M  25.7MB/s    in 2.1s    


In [8]:
#unzip datasets:

input_file = "reviews.csv.gz"
output_file = "reviews.csv"

with gzip.open(input_file, 'rb') as f_in:
    with open(output_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
print(f'O arquivo {input_file} foi descompactado para {output_file}.')


input_file2 = "listings.csv.gz"
output_file2 = "listings.csv"

with gzip.open(input_file2, 'rb') as f_in:
    with open(output_file2, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
print(f'O arquivo {input_file2} foi descompactado para {output_file2}.')

O arquivo reviews.csv.gz foi descompactado para reviews.csv.
O arquivo listings.csv.gz foi descompactado para listings.csv.


In [9]:
# Load real estate dataset
df_houses = spark.read.csv("listings.csv", header=True, inferSchema=True)

# Load review dataset
df_reviews = spark.read.csv("reviews.csv", header=True, inferSchema=True)

In [10]:
#houses
df_houses.show(3)

+----+--------------------+--------------+------------+---------------+--------------------+--------------------+---------------------+--------------------+-------+--------------------+---------+----------+-------------+--------------------+------------------+------------------+--------------------+-----------------+--------------------+--------------------+------------------+-------------------+-------------------------+--------------------+--------------------+----------------------+--------------------+----------------------+----------------------------+--------+---------+--------------------+---------------+------------+---------+--------------+--------+----+--------------------+--------------------+--------------------+--------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+--------------------+-----------------+--------------------+--------------------+---------------+----------

In [11]:
#rows
df_houses.count()

24182

In [12]:
#reviews
df_reviews.show(3)

+----------+-------+----------+-----------+-------------+--------------------+
|listing_id|     id|      date|reviewer_id|reviewer_name|            comments|
+----------+-------+----------+-----------+-------------+--------------------+
|      5456|    865|2009-03-08|       5267|        Ellen|Sylvia is a hoste...|
|    282342| 913203|2012-02-11|     633688|      Claudia|This is a fantast...|
|    282342|1064098|2012-03-31|    1613219|        Kerry|Chris and his fam...|
+----------+-------+----------+-----------+-------------+--------------------+
only showing top 3 rows



In [14]:
#rows
df_reviews.count()

583744

## Create Views

In [15]:
#create views from datasets
df_houses.createOrReplaceTempView("houses")
df_reviews.createOrReplaceTempView("reviews")

## 1 Eliminate properties without review

In [16]:
# Select 1 property: https://www.airbnb.com.br/rooms/5456

In [17]:
#SPARK SQL
spark.sql('''
          SELECT
            *
          FROM houses h
          WHERE h.id = 5456

          --WHERE host_name like '%Serita%'
        ''').show(10, truncate=False)

+----+---------------------------------+--------------+------------+-----------+----------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Select reviews from property

In [18]:
spark.sql('''
          SELECT
            *
          FROM reviews
          WHERE listing_id = 5456

          --WHERE EXTRACT (YEAR FROM date) = 2023
          --AND reviewer_name = 'Sneha'

        ''').show(10,truncate=False) #)

+----------+----+----------+-----------+------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|listing_id|id  |date      |reviewer_id|reviewer_name     |comments                                                                                                                                                                                                                                                                                                                                                            

In [20]:
#Create new dataset just w/ properties with review:

houses_w_review = spark.sql('''
          SELECT
            *
          FROM houses h
          WHERE h.id IN (SELECT listing_id FROM reviews)

        ''')

houses_w_review.show(5)

+--------+--------------------+--------------+------------+---------------+--------------------+--------------------+---------------------+--------------------+--------+--------------------+-----------+----------+-------------+--------------------+------------------+------------------+--------------------+-----------------+--------------------+--------------------+------------------+-------------------+-------------------------+--------------------+--------------------+----------------------+--------------------+----------------------+----------------------------+--------+---------+-------------+---------------+------------+---------+--------------+--------+----+--------------------+----------------+--------------------+--------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------+--------------------+--------------------+---------------+--------------------+---------------

In [21]:
#total number of houses = 14861
#total number of houses w/ review = 11758

In [22]:
#create view from dataset
houses_w_review.createOrReplaceTempView("houses_w_reviews")

In [23]:
#select from dataset created
spark.sql('''
          SELECT
           *
          FROM houses_w_reviews
        ''').show(10) #,truncate=False)

+--------+--------------------+--------------+------------+---------------+--------------------+--------------------+---------------------+--------------------+---------+--------------------+-----------+----------+--------------------+--------------------+------------------+------------------+--------------------+-----------------+--------------------+--------------------+------------------+-------------------+-------------------------+--------------------+--------------------+----------------------+--------------------+----------------------+----------------------------+--------+---------+--------------------+---------------+------------+---------+----------------+--------+----+--------------------+--------------------+--------------------+--------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+------------------+--------------------+--------------------+-----------------+-----------

## 2 Identify customers who were detractors in some evaluation

In [ ]:
#objective: identify from comments, which customers were detractors in some review

In [25]:
#extract evaluation from column name, and create new view with column evaluation:
houses_w_ratings = spark.sql('''
          SELECT
              id,
              name,
              regexp_extract(name, '★([0-9]*\.[0-9]+)', 1) AS evaluation,
              host_location,
              latitude,
              longitude,
              property_type,
              accommodates
          FROM
              houses_w_reviews
          WHERE
              name LIKE "%★%"
        ''')

#create view from dataset
houses_w_ratings.createOrReplaceTempView("houses_w_ratings")

In [26]:
#houses_w_ratings view:
spark.sql('''
          SELECT
             *
          FROM houses_w_ratings
        ''').show(10) #,truncate=False)

+--------+--------------------+----------+--------------------+--------+---------+--------------------+------------+
|      id|                name|evaluation|       host_location|latitude|longitude|       property_type|accommodates|
+--------+--------------------+----------+--------------------+--------+---------+--------------------+------------+
|17239710|Tiny home in Aust...|      4.95|          Austin, TX|    null|     null|                null|        null|
|18227140|Home in Austin · ...|      4.95|            Kyle, TX| 30.2205|-97.70055|         Entire home|          12|
|20932451|Home in Austin · ...|      4.98|          Austin, TX|30.25544|-97.73423|         Entire home|          16|
|21098269|Loft in Austin · ...|      4.95|          Austin, TX|30.26014| -97.7154|         Entire loft|           2|
| 2322667|Home in Austin · ...|       5.0|          Austin, TX|    null|     null|                null|        null|
|28302431|Townhouse in Aust...|       5.0|          Austin, TX|3

In [27]:
#identify properties with evaluation < 2, to get an example
spark.sql('''
          SELECT
             *
          FROM houses_w_ratings
          WHERE evaluation < 2 AND evaluation > 0
        ''').show(10) #,truncate=False)

+------------------+--------------------+----------+-------------+------------------+------------------+--------------------+------------+
|                id|                name|evaluation|host_location|          latitude|         longitude|       property_type|accommodates|
+------------------+--------------------+----------+-------------+------------------+------------------+--------------------+------------+
|844172257191676124|Home in Austin · ...|      1.40|         null|30.390417104953457|-97.68489815675012|Private room in home|           3|
+------------------+--------------------+----------+-------------+------------------+------------------+--------------------+------------+



In [28]:
#check property evaluations: id 844172257191676124

five_neg_comments = spark.sql('''
          SELECT
             *
          FROM reviews
          where listing_id = 844172257191676124
        ''')

five_neg_comments.show(10, truncate=False) #)

+------------------+------------------+----------+-----------+---------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|listing_id        |id                |date      |reviewer_id|reviewer_name  |comments                             

In [29]:
#these 5 customers above were detractors.
#Run sentiment analysis trying to identify them all, from their comments

### Sentiment Analysis

In [30]:
# Define a UDF (User Defined Function) for sentiment analysis

def analyze_sentiment(comment):
   # Check if comment is not None
    if comment is not None:
        analysis = TextBlob(comment)
        if analysis.sentiment.polarity > 0:
            return 'Positive'
        elif analysis.sentiment.polarity == 0:
            return 'Neutral'
        else:
            return 'Negative'
    else:
        return 'N/A'  # You can handle None values as needed

In [31]:
#test function with first comment:
analyze_sentiment("TERRIBLE TERRIBLE. I have been using Airbnb for some years now and have NEVER left a bad review. <br/>This is the first person to truly have poor customer service. From taking hours to send the information to get inside (which happened to include incorrect lock codes). The house is very run down. Showers crusty with no towels. Had to wait 2 full hours to get inside the house after he came to see what was going on. He refused to provide any partial refund after all the inconvenience. <br/>An absolute mess. I’d recommend paying an extra $5-10 and just getting a different airbnb within the same area to potentially save you some trouble and headaches. Hope this helped, I tried to be as genuine as possible. <br/>Airbnb support was extremely helpful with this poor situation so I am grateful for them.")


'Negative'

In [34]:
#worked

#create new column in df_reviews
df_reviews.show(3)

+----------+-------+----------+-----------+-------------+--------------------+
|listing_id|     id|      date|reviewer_id|reviewer_name|            comments|
+----------+-------+----------+-----------+-------------+--------------------+
|      5456|    865|2009-03-08|       5267|        Ellen|Sylvia is a hoste...|
|    282342| 913203|2012-02-11|     633688|      Claudia|This is a fantast...|
|    282342|1064098|2012-03-31|    1613219|        Kerry|Chris and his fam...|
+----------+-------+----------+-----------+-------------+--------------------+
only showing top 3 rows



In [35]:
#In PySpark, a User Defined Function (UDF) allows you to define a custom function in Python and apply it to columns in a PySpark DataFrame

# Register the UDF with Spark
sentiment_udf = udf(analyze_sentiment, StringType()) #StringType: specifies the return type of the UDF, was set string to return 'Positive', 'Neutral', or 'Negative'

In [36]:
#Once the UDF is registered, you can use it with the withColumn method to apply the sentiment analysis function to a specific column in a PySpark DataFrame.
df_reviews_w_sentiment = df_reviews.withColumn('sentiment', sentiment_udf(df_reviews['comments']))

In [37]:
#New dataframe with sentiments:
df_reviews_w_sentiment.show(3)

+----------+-------+----------+-----------+-------------+--------------------+---------+
|listing_id|     id|      date|reviewer_id|reviewer_name|            comments|sentiment|
+----------+-------+----------+-----------+-------------+--------------------+---------+
|      5456|    865|2009-03-08|       5267|        Ellen|Sylvia is a hoste...| Positive|
|    282342| 913203|2012-02-11|     633688|      Claudia|This is a fantast...| Positive|
|    282342|1064098|2012-03-31|    1613219|        Kerry|Chris and his fam...| Positive|
+----------+-------+----------+-----------+-------------+--------------------+---------+
only showing top 3 rows



In [38]:
#Create new view with sentiment:
df_reviews_w_sentiment.createOrReplaceTempView("reviews_w_sentiments")

In [39]:
#check those 5 negative coments, if were identified as negative:
spark.sql('''
          SELECT
             *
          FROM reviews_w_sentiments
          where listing_id = 844172257191676124
        ''').show(10, truncate=False) #)

+------------------+------------------+----------+-----------+---------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+
|listing_id        |id                |date      |reviewer_id|reviewer_name  |comments                   

In [ ]:
#Interesing stuff:
  # Positive: Second was kind of ironic, not mentioning negative words, and was classified as positive
  # Neutral: Fifth was neutral, but it's Spanish
  # Negative: 3 others

In [60]:
#check those 5 negative coments, if were identified as negative:
spark.sql('''
          SELECT
             *
          FROM reviews_w_sentiments
          where listing_id = 844172257191676124
        ''').show(10, truncate=False) #)

KeyboardInterrupt: ignored

In [61]:
#create new column in df_reviews
df_reviews.show(3)

KeyboardInterrupt: ignored

In [ ]:
# 500 k + rows, to much to execute in Colab. --> LIMIT NUM OF ROWS!

In [56]:
spark.sql('''
          SELECT
            *
          FROM reviews_w_sentiments
          limit 100
         -- WHERE sentiment IN ('Positive', 'Neutral', 'Negative')
        ''').show(10) #, truncate=False

KeyboardInterrupt: ignored

In [40]:
#count number of negative, neutral and positive:
df_reviews_w_sentiment.groupby("sentiment").count().show()  #more than min and nothing

KeyboardInterrupt: ignored

## 3 Get bag of words from all negative reviews

# Melhorias próximos ciclos:
-